# Discussion 05 Notebook

This notebook is an accompaniment to the associated discussion worksheet handout.

## Load in the IMDB Performance database

This is a variation of the IMDB database with keys defined. Note that this is a pretty big database! So if you run the below lines, please also remember to delete the `imdb_perf_lecture` afterwards to save space on your limited postgreSQL server.

If you didn't load it in with a previous lecture, load in the `imdb_perf_lecture` database.

In [1]:
!ln -sf ../../lec/lec06/data .
!unzip -u data/imdb_perf_lecture.zip -d data/

Archive:  data/imdb_perf_lecture.zip
  inflating: data/imdb_perf_lecture.sql  


In [2]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS imdb_perf_lecture'
!psql -h localhost -c 'CREATE DATABASE imdb_perf_lecture' 
!psql -h localhost -d imdb_perf_lecture -f data/imdb_perf_lecture.sql

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 845888
COPY 2211936
COPY 656453
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE


In [3]:
%reload_ext sql
%sql postgresql://127.0.0.1:5432/imdb_perf_lecture
import pandas as pd


Before starting this part, review the schema of the relations in the `imdb_perf_lecture` database. Here's the printout from `psql`:

```
imdb_perf_lecture=# \d actor
               Table "public.actor"
 Column |  Type   | Collation | Nullable | Default 
--------+---------+-----------+----------+---------
 id     | integer |           | not null | 
 name   | text    |           |          | 
Indexes:
    "actor_pkey" PRIMARY KEY, btree (id)
Referenced by:
    TABLE "cast_info" CONSTRAINT "cast_info_person_id_fkey" FOREIGN KEY (person_id) REFERENCES actor(id)

imdb_perf_lecture=# \d cast_info
               Table "public.cast_info"
  Column   |  Type   | Collation | Nullable | Default 
-----------+---------+-----------+----------+---------
 person_id | integer |           |          | 
 movie_id  | integer |           |          | 
Foreign-key constraints:
    "cast_info_movie_id_fkey" FOREIGN KEY (movie_id) REFERENCES movie(id)
    "cast_info_person_id_fkey" FOREIGN KEY (person_id) REFERENCES actor(id)

imdb_perf_lecture=# \d movie
                    Table "public.movie"
     Column      |  Type   | Collation | Nullable | Default 
-----------------+---------+-----------+----------+---------
 id              | integer |           | not null | 
 title           | text    |           |          | 
 year            | integer |           |          | 
 runtime_minutes | integer |           |          | 
Indexes:
    "movie_pkey" PRIMARY KEY, btree (id)
Referenced by:
    TABLE "cast_info" CONSTRAINT "cast_info_movie_id_fkey" FOREIGN KEY (movie_id) REFERENCES movie(id)
```

# Section III

## Question 8

Write a query that computes an inner join on `actors` and `cast_info` on the actor ID. Your query should return all attributes.

In [12]:
%%sql
-- write your query here --


Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

RuntimeError: (psycopg2.ProgrammingError) can't execute an empty query
[SQL: -- write your query here --]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


Once you're comfortable that the query is working as expected, run it through `EXPLAIN ANALYZE`. **Discuss**: What kind of join is the query optimizer performing? Why might this be the case?

In [13]:
%%sql
-- write your EXPLAIN ANALYZE here --


Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

RuntimeError: (psycopg2.ProgrammingError) can't execute an empty query
[SQL: -- write your EXPLAIN ANALYZE here --]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


<br/>

---

Sometimes you may prefer to adjust the PostgreSQL settings to force a specific performance of a query. 
In the remainder of this question we will explore how to specify these settings. Note that settings tweaking for a single query is **not recommended** in practice, as it affects all of your queries! However, if you know specific characteristics of how your database will be queried in general, then by all means, dive in to adjust these knobs.

* All runtime parameters for PostgreSQL are in one view, called `pg_settings` (Documentation 54.24 [link](https://www.postgresql.org/docs/current/view-pg-settings.html)).
* In particular, the Planner Method Configuration (Documentation 20.7.1 [link](https://www.postgresql.org/docs/current/runtime-config-query.html#GUC-SEQ-PAGE-COST)) includes the parameter descriptions for the query optimizer.

We encourage you to keep these pages up as you explore the next activity.

## Question 9

Run the below query. Which settings are related to selecting the type of join that the query optimizer can select?

In [8]:
%%sql
SELECT name
FROM pg_settings
WHERE name LIKE 'enable_%';


Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

20 rows affected.

name
enable_async_append
enable_bitmapscan
enable_gathermerge
enable_hashagg
enable_hashjoin
enable_incremental_sort
enable_indexonlyscan
enable_indexscan
enable_material
enable_memoize


## Question 10

Let's suppose we turn off hash join as an option for the query optimizer. The syntax is included for you below.

In [19]:
# just run this cell
%sql set enable_hashjoin=false;

Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

++
||
++
++

Copy your `EXPLAIN ANALYZE` command from Question 8 and rerun it below.

**Discuss**: Recall that our intial query was performed using hash joins; what is the join the query optimizer picks below? Why might this be preferred over the remaining alternatives?

In [18]:
%%sql
-- copy your EXPLAIN ANALYZE query from the previous part here --


Running query in 'postgresql://127.0.0.1:5432/imdb_perf_lecture'

RuntimeError: (psycopg2.ProgrammingError) can't execute an empty query
[SQL: -- copy your EXPLAIN ANALYZE query from the previous part here --]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


## Question 11

Next, what if we turn off the option for using the join in Question 10? In the cell below, replace `# YOUR CODE HERE` with the one-line SQL magic that will set the corresponding `pg_settings` entry to `false`.

In [17]:
# YOUR SQL line MAGIC HERE


Copy your `EXPLAIN ANALYZE` command from the previous parts and rerun it below. Note the selected join, as well as the significnatly longer execution time!

In [ ]:
%%sql
-- write your query here; edit to include EXPLAIN ANALYZE --


## Question 12

**Cleanup** Finally, reset the two settings you edited in Question 10 and 11 in the cell below.

In [ ]:
# YOUR SQL line MAGIC HERE
# YOUR SQL line MAGIC HERE
